# HW5 - Predictive Regression
Bailey Meche

This notebook `hw5_baileymeche.ipynb` executes all files in this [repo](https://github.com/BaileyMeche/qts/tree/main/HW5). Inspect individual files for any specific code. The report below reports all findings.

# Report


**Methods, Results, and Robustness (Boxcar OLS vs Exponentially Weighted LS)**

## 1. Objective and required deliverables

This project implements the assignment’s two-stage framework:

1. **Construct weekly (Wednesday-to-Wednesday) return series** for:

   * Single-name **5Y CDS spreads** (treated as “prices” for percent returns),
   * Single-name **equity adjusted closes**,
   * **Market** returns via **SPY**.
2. **Contemporaneous (hedging) regressions** (rolling **boxcar OLS**, window (K=16)):

   * Equity CAPM: $r^{Equity}_{E,n} \sim m_n \to \gamma_{E,n}$
   * CDS model: $r^{CDS}_{E,n} \sim r^{Equity}*{E,n} + r^{Index}_n \to \beta^{(n)}_{E,Equity}, \beta^{(n)}_{E,Index}$
3. Build:

   * Hedge return: $f_{E,n}=\beta^{(n)}_{E,Equity}r^{Equity}_{E,n}+\beta^{(n)}_{E,Index}r^{Index}_n$
   * Hedged CDS residual: $\rho_{E,n}=r^{CDS}*{E,n}-f*{E,n}$
   * Residual equity return: $c_{E,n}=r^{Equity}*{E,n}-\gamma*{E,n}m_n$
4. **Predictive regression** (strict lag, no lookahead):
   $$
   \rho_{E,n} \sim c_{E,n-1}
   $$
   Compare:

   * **Boxcar OLS** (window 16),
   * **Exponentially weighted / discounted LS** (**half-life 12**; window 16).
     Form predictive residuals:
     $$
     q_{E,n}=\rho_{E,n}-\mu_{E,n-1}c_{E,n-1}
     $$
5. **Analysis**: compare performance across time (“events”), tail behavior, and statistical properties. No trading strategy required.

---

## 2. Data and construction

### 2.1 Instruments and sample

* **Tickers:** `AES, BA, C, F, GE, JPM, LNC, LOW, LUV, MAR, MCD, NFLX, T, WFC, WHR, XOM, XRX`
* **Frequency:** Weekly, anchored **Wednesday-to-Wednesday**, using the last available observation in each `W-WED` bin.
* **Sample range (weekly):** **2018-01-03** to **2025-12-31** (**397 weeks**).

### 2.2 Returns

* **Equity and SPY:** simple percent return on weekly adjusted close.
* **CDS “returns”:** simple percent return on weekly CDS parspread levels (treated like a price series as required).

### 2.3 CDS “Index return”

For each week (n):
[
r^{Index}*n=\frac{1}{N}\sum*{E=1}^N r^{CDS}_{E,n}
]

### 2.4 No-lookahead / strict lag alignment

All rolling regression coefficients used at week $n$ are estimated using **only prior weeks**, and the predictive residual definition uses $\mu_{n-1}$ and $c_{n-1}$ when forecasting $\rho_n$, matching the assignment’s indexing example. 

---

## 3. Model specification and estimation

### 3.1 Contemporaneous hedging regressions (boxcar OLS, (K=16))

For each ticker (E):

**Equity CAPM (rolling):**
$$
r^{Equity}*{E,n}\sim \gamma*{E,n} m_n
$$

**CDS contemporaneous model (rolling):**
$$
r^{CDS}*{E,n}\sim \beta^{(n)}*{E,Equity} r^{Equity}*{E,n}+\beta^{(n)}*{E,Index} r^{Index}_n
$$

### 3.2 Construct hedged series

$$
f_{E,n}=\beta^{(n)}*{E,Equity}r^{Equity}*{E,n}+\beta^{(n)}*{E,Index}r^{Index}*n
$$

$$
\rho*{E,n}=r^{CDS}*{E,n}-f_{E,n}
$$

$$
c_{E,n}=r^{Equity}*{E,n}-\gamma*{E,n}m_n
$$

### 3.3 Predictive regression (compare two estimators)

**Predictive model:**
$$
\rho_{E,n}\sim \mu_{E,n} c_{E,n-1}
$$

**Estimators compared:**

* **Boxcar OLS** (window 16, **no intercept** — consistent with the HW’s definitions of $f_{E,n}, c_{E,n}$, and $q_{E,n}$, which contain no constant term),
* **Exponentially weighted LS** (half-life 12; window 16; weights renormalized after NaN-masking; **no intercept** for the same reason).

**Intercept robustness check (not required by the HW):** I also reran the predictive regression with an intercept and confirmed conclusions were unchanged (Boxcar vs EWLS ranking and event/tail behavior).

**Predictive residuals:**
$$
q_{E,n}=\rho_{E,n}-\mu_{E,n-1}c_{E,n-1}
$$

---

## 4. Outputs produced

Generated artifacts (from `HW5_spine` run):

* `panel_results.csv` (full panel with $r^{CDS}, r^{Equity}, m, r^{Index}, \gamma, \beta, f, \rho, c, \mu, \hat\rho, q$)
* `metrics_per_ticker.csv`
* `metrics_pooled.csv`
* `event_windows.csv`
* Figures: `rolling_rmse.png`, `error_gap.png`, `tail_qq.png`
* Robustness: `robustness_sweep.csv`, `delta_rmse_heatmap.png`, `stability_heatmap.png`
  Interpretation scaffold: 

---

## 5. Core results (required comparison: Boxcar vs EWLS with half-life 12)

### 5.1 Pooled performance (all tickers pooled)

`metrics_pooled.csv` reports pooled summary statistics of predictive errors (q).

| model  |     rmse |    oos_r2 | rmse_delta_vs_boxcar | oos_r2_delta_vs_boxcar |       q01 |       q05 |      q95 |      q99 |     skew |   kurtosis |           n |
| :----- | -------: | --------: | -------------------: | ---------------------: | --------: | --------: | -------: | -------: | -------: | ---------: | ----------: |
| boxcar | 0.078726 | -0.241570 |             0.000000 |               0.000000 | -0.164503 | -0.071141 | 0.074031 | 0.167206 | 1.655034 | 216.677998 | 6274.000000 |
| ew     | 0.081275 | -0.323290 |             0.002550 |              -0.081720 | -0.165086 | -0.070221 | 0.073929 | 0.167253 | 0.421885 | 229.927242 | 6274.000000 |

**Key pooled findings (assignment setting: window 16, half-life 12):**

* **Boxcar OLS has lower pooled RMSE** than EWLS by **0.00255**.
* EWLS has **more negative** pooled out-of-sample (R^2) (delta **−0.0817**).
* **Tail quantiles are very similar** at pooled level (q01/q99 are nearly identical), while **shape** differs:

  * EWLS error distribution exhibits **lower skew** but **higher kurtosis**.

### 5.2 Weekly “event” behavior (historical windows of divergence)

Define weekly pooled RMSE for each method as:
$$
RMSE_n=\sqrt{\mathbb{E}*E[q*{E,n}^2]}
$$
and analyze the weekly gap $RMSE^{EW}_n-RMSE^{Box}_n$.

Across **397** weeks:

* Fraction of weeks EWLS is better (gap < 0): **47.85%**
* Fraction of weeks Boxcar is better (gap > 0): **47.13%**
* Mean gap: **+0.000255** (EWLS slightly worse on average)
* Mean gap excluding the single worst week: **+0.000028** (nearly zero)
* Largest EWLS underperformance week gap: **+0.09001**
* Largest EWLS outperformance week gap: **−0.00745**

**Largest EWLS underperformance windows (EW − Box):**

| date       | rmse_boxcar |  rmse_ew | gap_ew_minus_boxcar |
| :--------- | ----------: | -------: | ------------------: |
| 2020-03-25 |    0.607955 | 0.697966 |            0.090011 |
| 2020-04-01 |    0.436777 | 0.471505 |            0.034728 |
| 2020-04-08 |    0.158247 | 0.178788 |            0.020541 |
| 2020-03-18 |    0.421010 | 0.427715 |            0.006705 |
| 2020-11-18 |    0.058870 | 0.064301 |            0.005431 |

**Largest EWLS outperformance windows (EW − Box):**

| date       | rmse_boxcar |  rmse_ew | gap_ew_minus_boxcar |
| :--------- | ----------: | -------: | ------------------: |
| 2025-07-30 |    0.044290 | 0.036841 |           -0.007449 |
| 2025-08-06 |    0.044477 | 0.037674 |           -0.006804 |
| 2022-02-02 |    0.059398 | 0.053129 |           -0.006269 |
| 2020-09-16 |    0.064941 | 0.059771 |           -0.005170 |
| 2025-02-19 |    0.031439 | 0.026944 |           -0.004495 |

**Interpretation:** The most material divergence occurs during **March–April 2020**, where EWLS performs substantially worse than boxcar. Outside such stress windows, differences are typically small and frequently switch sign.

### 5.3 Tail behavior (required: “How are the tails?”)

* Pooled q01/q99 are extremely close between methods (Section 5.1).
* The **tail QQ plot** indicates near one-for-one mapping between quantiles of EWLS and boxcar errors (i.e., tail behavior is broadly similar in quantile space), while higher moments differ (skew/kurtosis).

---

## 6. Per-ticker results (required granularity)

Table below reports per-ticker RMSE and OOS (R^2) for both estimators and the RMSE difference (EW − Box). (GE did not produce valid metrics in the output; NFLX has fewer observations.)

| Ticker | RMSE_Boxcar | RMSE_EWLS | ΔRMSE (EW-Box) | OOS_R2_Boxcar | OOS_R2_EWLS |   n |
| :----- | ----------: | --------: | -------------: | ------------: | ----------: | --: |
| AES    |    0.104287 |  0.113920 |       0.009633 |     -2.415728 |   -3.075912 | 397 |
| BA     |    0.171197 |  0.179530 |       0.008333 |     -0.322178 |   -0.454023 | 397 |
| C      |    0.044975 |  0.045745 |       0.000770 |     -0.158383 |   -0.198373 | 397 |
| F      |    0.069517 |  0.071609 |       0.002092 |     -0.110431 |   -0.178271 | 397 |
| GE     |         NaN |       NaN |            NaN |           NaN |         NaN | NaN |
| JPM    |    0.044125 |  0.045037 |       0.000911 |     -0.105452 |   -0.151587 | 397 |
| LNC    |    0.069537 |  0.070061 |       0.000524 |     -0.222138 |   -0.240640 | 397 |
| LOW    |    0.060457 |  0.060468 |       0.000011 |      0.009579 |    0.009204 | 397 |
| LUV    |    0.108622 |  0.108906 |       0.000284 |      0.031867 |    0.026801 | 397 |
| MAR    |    0.074054 |  0.076521 |       0.002467 |     -0.157370 |   -0.235774 | 397 |
| MCD    |    0.062312 |  0.066005 |       0.003693 |     -0.367048 |   -0.533878 | 397 |
| NFLX   |    0.066239 |  0.065716 |      -0.000523 |     -0.104937 |   -0.087569 | 319 |
| T      |    0.036987 |  0.037505 |       0.000518 |     -0.104770 |   -0.135955 | 397 |
| WFC    |    0.049330 |  0.049928 |       0.000599 |     -0.111594 |   -0.138739 | 397 |
| WHR    |    0.051731 |  0.051760 |       0.000029 |     -0.035720 |   -0.036882 | 397 |
| XOM    |    0.053945 |  0.053995 |       0.000051 |     -0.036471 |   -0.038413 | 397 |
| XRX    |    0.079938 |  0.079581 |      -0.000357 |     -0.036557 |   -0.027322 | 397 |

**Per-ticker summary:**

* EWLS (half-life 12) improves RMSE for **2/16** tickers with valid comparisons (**NFLX**, **XRX**), is roughly neutral for a few (e.g., LOW), and is worse for most tickers.
* The largest EWLS deterioration is in **AES** and **BA** (RMSE deltas (\approx) 0.008–0.010).
* **GE** metrics are missing in the provided output (diagnosis should be tied to missingness / alignment in the underlying time series for that ticker).

---

## 7. Statistical properties: what differs between estimators?

Based on pooled error distributions (Section 5.1):

* **Location/scale:** Similar mean and standard deviation; boxcar has slightly smaller RMSE.
* **Tails (quantiles):** q01/q99 are nearly identical between methods.
* **Shape:** EWLS produces **substantially lower skew** but **higher kurtosis**.

Interpretation consistent with time-local behavior:

* EWLS can track changes more rapidly but may be **more fragile in abrupt regime shifts**, as reflected by the large March–April 2020 spike in the weekly RMSE gap.

---

## 8. Above-and-beyond (explicitly noted)

### 8.1 Robustness / sensitivity sweep (not required by assignment)

A sensitivity analysis was run over:

* Predictive windows: **{12, 16, 26}**
* EWLS half-lives: **{8, 12, 20}**

Results (`robustness_sweep.csv`) show:

* Baseline (assignment EWLS): **window 16, half-life 12**, RMSE = **0.081275**
* Best-performing EWLS configuration in the sweep:

  * **window 26, half-life 20**, RMSE = **0.076734**
  * This improves RMSE by **0.004541** versus the baseline EWLS setting
  * And is **0.00199 lower** than boxcar’s pooled RMSE (**0.078726**)

Stability (ticker-ranking correlation of RMSE deltas) is consistently high (**~0.98–1.00**), indicating that relative ticker impacts of parameter changes are stable.

**Full sweep table (EWLS only):**

| window | half_life |  ew_rmse | ew_oos_r2 | delta_rmse_vs_baseline | delta_r2_vs_baseline | stability_rank_corr |
| -----: | --------: | -------: | --------: | ---------------------: | -------------------: | ------------------: |
|     12 |       8.0 | 0.083703 | -0.403537 |               0.002428 |            -0.080247 |            0.982353 |
|     12 |      12.0 | 0.082573 | -0.365878 |               0.001297 |            -0.042587 |            1.000000 |
|     12 |      20.0 | 0.081710 | -0.337479 |               0.000435 |            -0.014189 |            0.991176 |
|     16 |       8.0 | 0.082601 | -0.366819 |               0.001326 |            -0.043528 |            0.985294 |
|     16 |      12.0 | 0.081275 | -0.323290 |               0.000000 |             0.000000 |            1.000000 |
|     16 |      20.0 | 0.080237 | -0.289702 |              -0.001038 |             0.033588 |            1.000000 |
|     26 |       8.0 | 0.080206 | -0.274193 |              -0.001069 |             0.049097 |            0.994118 |
|     26 |      12.0 | 0.078223 | -0.211971 |              -0.003052 |             0.111319 |            1.000000 |
|     26 |      20.0 | 0.076734 | -0.166268 |              -0.004541 |             0.157022 |            0.997059 |

**Takeaway:** Under the assignment’s fixed half-life (12), EWLS is slightly worse than boxcar in pooled RMSE; however, **stronger smoothing** (longer window and longer half-life) can materially improve EWLS and even outperform boxcar in this dataset.

---

## 9. Figures (placeholders for insertion)

Insert the generated figures below (filenames correspond to the run outputs):

### Figure 1 — Rolling pooled RMSE (12-week mean)

![Rolling pooled RMSE (12-week mean): Boxcar vs EWLS](output/rolling_rmse.png)

### Figure 2 — Weekly pooled RMSE gap (EWLS − Boxcar)

![Weekly pooled RMSE gap (EWLS − Boxcar)](output/error_gap.png)

### Figure 3 — Tail comparison (error quantile–quantile)

![Tail QQ: EWLS vs Boxcar error quantiles](output/tail_qq.png)

### Figure 4 — Robustness sweep: ΔRMSE vs baseline (EWLS)

![Robustness heatmap: delta\_rmse\_vs\_baseline](output/robustness/delta_rmse_heatmap.png)

### Figure 5 — Robustness sweep: stability (rank correlation)

![Robustness heatmap: stability\_rank\_corr](output/robustness/stability_heatmap.png)

---

## 10. Reproducibility and audit trail

* The complete computation (data → weekly construction → rolling regressions → predictive regression → outputs) is executed in `HW5_spine.ipynb`.
* The full panel used to compute all summary tables/plots is stored in `panel_results.csv`, enabling direct verification of:

  * Weekly series construction $r^{CDS}, r^{Equity}, m, r^{Index}$,
  * Rolling coefficients $\gamma, \beta$,
  * Hedged residuals $\rho$ and residual equity returns $c$,
  * Predictive coefficients $\mu$ and predictive residuals $q$.

---

## Output files and notes:

* `metrics_pooled.csv`
* `metrics_per_ticker.csv`
* `event_windows.csv`
* `robustness_sweep.csv`
* `panel_results.csv`
* Figures: `rolling_rmse.png`, `error_gap.png`, `tail_qq.png`, `delta_rmse_heatmap.png`, `stability_heatmap.png`

* The assignment comparison is performed exactly at **window 16** and **EWLS half-life 12**; alternative EWLS smoothness settings materially change conclusions (Section 8).
* Some tickers may have incomplete aligned histories (e.g., **GE** metrics missing; **NFLX** has fewer observations), which affects pooled counts and per-ticker comparability.


# Code:

In [19]:
from __future__ import annotations

import importlib.util
import sys
from pathlib import Path
import pandas as pd
import warnings
import subprocess
from IPython.display import display
import re

def find_hw5_root(start: Path) -> Path:
    """Locate the HW5 project directory whether you opened this notebook from repo root or inside HW5/."""
    start = start.resolve()
    for p in [start] + list(start.parents):
        # Case 1: notebook run from HW5/ (preferred)
        if (p / "hw5").is_dir() and (p / "tests").is_dir() and (p / "Liq5YCDS.delim").exists():
            return p
        # Case 2: notebook run from repo root that contains HW5/
        if (p / "HW5" / "hw5").is_dir() and (p / "HW5" / "tests").is_dir() and (p / "HW5" / "Liq5YCDS.delim").exists():
            return p / "HW5"
    raise FileNotFoundError(
        "Could not locate HW5 project root. Run from repo root (containing HW5/) or from inside HW5/."
    )

HW5_ROOT = find_hw5_root(Path.cwd())
PROJECT_ROOT = HW5_ROOT.parent  # repo root (may be the same as HW5_ROOT if already at top)

# Ensure 'import hw5' works regardless of CWD
if str(HW5_ROOT) not in sys.path:
    sys.path.insert(0, str(HW5_ROOT))

print(f"Notebook CWD: {Path.cwd().resolve()}")
print(f"HW5_ROOT: {HW5_ROOT}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print("Python:", sys.version.split()[0])

# Dependency sanity checks (pyarrow is required to read/write parquet)
missing = [pkg for pkg in ["pyarrow"] if importlib.util.find_spec(pkg) is None]
if missing:
    req = HW5_ROOT / "requirements.txt"
    print("\nMissing required packages:", missing)
    if req.exists():
        print(f"Install with: {sys.executable} -m pip install -r {req}")
    else:
        print(f"Install with: {sys.executable} -m pip install " + " ".join(missing))

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)


Notebook CWD: C:\Users\Owner\Downloads\HW5
HW5_ROOT: C:\Users\Owner\Downloads\HW5
PROJECT_ROOT: C:\Users\Owner\Downloads
Python: 3.13.9


## 1) Run unit tests

In [20]:
tests_dir = HW5_ROOT / "tests"
cmd = [sys.executable, "-m", "pytest", str(tests_dir), "-q"]
print("Running:", " ".join(cmd))

res = subprocess.run(cmd, cwd=str(HW5_ROOT), capture_output=True, text=True)
print(res.stdout)

if res.returncode != 0:
    print("--- STDERR ---")
    print(res.stderr)
    print(f"pytest exited with code {res.returncode}")
else:
    print("pytest passed")


Running: c:\ProgramData\anaconda3\envs\risk\python.exe -m pytest C:\Users\Owner\Downloads\HW5\tests -q
.....                                                                    [100%]
5 passed in 0.71s

pytest passed


## 2) Run full HW5 pipeline

In [ ]:
FORCE_DOWNLOAD = False  
has_pyarrow = importlib.util.find_spec("pyarrow") is not None

if not has_pyarrow and not FORCE_DOWNLOAD:
    req = HW5_ROOT / "requirements.txt"
    print("pyarrow is required to read the included equity_adj_close.parquet cache.")
    if req.exists():
        print(f"Install with: {sys.executable} -m pip install -r {req}")
    else:
        print(f"Install with: {sys.executable} -m pip install pyarrow")
    print("\nIf you cannot install pyarrow but you have internet access, set FORCE_DOWNLOAD=True to fetch data via yfinance.")
else:
    from hw5.run_all import main
    main()
    print(f"Pipeline run complete. Outputs written to: {HW5_ROOT / 'output'}")



Pipeline run complete. Outputs written to: C:\Users\Owner\Downloads\HW5\output


## 3) Inspect generated artifacts

In [22]:
output_dir = HW5_ROOT / "output"
if output_dir.exists():
    for p in sorted(output_dir.rglob("*")):
        if p.is_file():
            print(p.relative_to(HW5_ROOT))
else:
    print("No output directory found yet. Run the pipeline cell above.")


output\error_gap.png
output\event_windows.csv
output\metrics_per_ticker.csv
output\metrics_pooled.csv
output\panel_results.csv
output\robustness\delta_rmse_heatmap.png
output\robustness\robustness_sweep.csv
output\robustness\stability_heatmap.png
output\rolling_rmse.png
output\summary.md
output\tail_qq.png


## 4) Display key metric tables

In [23]:
pooled_path = HW5_ROOT / "output" / "metrics_pooled.csv"
ticker_path = HW5_ROOT / "output" / "metrics_per_ticker.csv"
events_path = HW5_ROOT / "output" / "event_windows.csv"
robust_path = HW5_ROOT / "output" / "robustness" / "robustness_sweep.csv"

if pooled_path.exists():
    print("Pooled metrics")
    display(pd.read_csv(pooled_path))

if ticker_path.exists():
    print("Per ticker metrics (head)")
    display(pd.read_csv(ticker_path).head(20))

if events_path.exists():
    print("Largest event windows by RMSE gap")
    display(pd.read_csv(events_path).head(15))

if robust_path.exists():
    print("Robustness sweep")
    display(pd.read_csv(robust_path))


Pooled metrics


,rmse,mean,std,q01,q05,q95,q99,skew,kurtosis,n,model,oos_r2,rmse_delta_vs_boxcar,oos_r2_delta_vs_boxcar
0,0.078726,-0.000406,0.078731,-0.164503,-0.071141,0.074031,0.167206,1.655034,216.677998,6274.0,boxcar,-0.24157,0.00000,0.00000
1,0.081275,-0.000463,0.081280,-0.165086,-0.070221,0.073929,0.167253,0.421885,229.927242,6274.0,ew,-0.32329,0.00255,-0.08172


Per ticker metrics (head)


,ticker,model,rmse,mean,std,q01,q05,q95,q99,skew,kurtosis,n,oos_r2
0,AES,boxcar,0.104287,0.001194,0.104412,-0.109094,-0.064485,0.069826,0.158856,1.655508,108.386025,397.0,-2.415728
1,AES,ew,0.113920,0.001023,0.114060,-0.109211,-0.064796,0.069040,0.159304,0.473679,120.541956,397.0,-3.075912
2,BA,boxcar,0.171197,-0.000597,0.171412,-0.194823,-0.097338,0.098189,0.215415,3.443047,113.995513,397.0,-0.322178
3,BA,ew,0.179530,-0.001698,0.179749,-0.205471,-0.099333,0.094973,0.215563,1.615779,111.975329,397.0,-0.454023
4,C,boxcar,0.044975,-0.000889,0.045023,-0.085645,-0.046541,0.053048,0.092860,-3.213706,34.384909,397.0,-0.158383
5,C,ew,0.045745,-0.000907,0.045794,-0.086343,-0.045819,0.053019,0.092869,-3.088893,34.101004,397.0,-0.198373
6,F,boxcar,0.069517,0.000440,0.069603,-0.144828,-0.078477,0.076012,0.138717,4.785801,60.605326,397.0,-0.110431
7,F,ew,0.071609,0.000185,0.071699,-0.145761,-0.078433,0.075551,0.138818,4.050369,58.232877,397.0,-0.178271
8,GE,boxcar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GE,ew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Largest event windows by RMSE gap


,date,rmse_boxcar,rmse_ew,gap_ew_minus_boxcar
0,2020-03-25,0.607955,0.697966,0.090011
1,2020-04-01,0.436777,0.471505,0.034728
2,2020-04-08,0.158247,0.178788,0.020541
3,2025-07-30,0.044290,0.036841,-0.007449
4,2025-08-06,0.044477,0.037674,-0.006804
5,2020-03-18,0.421010,0.427715,0.006705
6,2022-02-02,0.059398,0.053129,-0.006269
7,2020-11-18,0.058870,0.064301,0.005431
8,2020-09-16,0.064941,0.059771,-0.005170
9,2025-02-19,0.031439,0.026944,-0.004495


Robustness sweep


,window,half_life,ew_rmse,ew_oos_r2,delta_rmse_vs_baseline,delta_r2_vs_baseline,stability_rank_corr
0,12,8.0,0.083703,-0.403537,0.002428,-0.080247,0.982353
1,12,12.0,0.082573,-0.365878,0.001297,-0.042587,1.000000
2,12,20.0,0.081710,-0.337479,0.000435,-0.014189,0.991176
3,16,8.0,0.082601,-0.366819,0.001326,-0.043528,0.985294
4,16,12.0,0.081275,-0.323290,0.000000,0.000000,1.000000
5,16,20.0,0.080237,-0.289702,-0.001038,0.033588,1.000000
6,26,8.0,0.080206,-0.274193,-0.001069,0.049097,0.994118
7,26,12.0,0.078223,-0.211971,-0.003052,0.111319,1.000000
8,26,20.0,0.076734,-0.166268,-0.004541,0.157022,0.997059


In [24]:
panel_path = HW5_ROOT / "output" / "panel_results.csv"
df = pd.read_csv(panel_path, parse_dates=["date"])
df = df.sort_values(["ticker", "date"]).reset_index(drop=True)

def pick(cols, patterns):
    for pat in patterns:
        r = re.compile(pat)
        hit = [c for c in cols if r.fullmatch(c) or r.search(c)]
        if hit:
            return hit[0]
    raise KeyError(f"Could not find any of patterns={patterns} in columns")

cols = df.columns

col_rho = pick(cols, [r"\brho\b", r"rho_"])
col_c   = pick(cols, [r"\bc\b", r"c_"])
col_f   = pick(cols, [r"\bf\b", r"f_"])
col_rcds= pick(cols, [r"r.*cds", r"cds_return", r"r_cds"])
col_req = pick(cols, [r"r.*equity", r"equity_return", r"r_equity"])
col_m   = pick(cols, [r"\bm\b", r"market", r"spy"])
col_gamma = pick(cols, [r"gamma", r"\bgamma_"])
# beta columns (optional)
beta_eq = [c for c in cols if "beta" in c.lower() and "equity" in c.lower()]
beta_ix = [c for c in cols if "beta" in c.lower() and ("index" in c.lower() or "idx" in c.lower())]

g = df.groupby("ticker", sort=False)

# --- Identity checks (must hold up to floating error) ---
rho_recalc = df[col_rcds] - df[col_f]
c_recalc   = df[col_req] - df[col_gamma] * df[col_m]

rho_err = (df[col_rho] - rho_recalc).abs()
c_err   = (df[col_c]   - c_recalc).abs()

# --- Predictive lag check: q == rho - mu_{t-1} * c_{t-1} ---
# Try to locate mu/q for boxcar + ewls (or whatever you named them)
mu_cols = [c for c in cols if re.search(r"\bmu\b", c.lower())]
q_cols  = [c for c in cols if re.search(r"\bq\b", c.lower())]

def report_predictive_check(mu_col, q_col, label):
    mu_lag = g[mu_col].shift(1)
    c_lag  = g[col_c].shift(1)
    q_recalc = df[col_rho] - mu_lag * c_lag
    q_err = (df[q_col] - q_recalc).abs()
    out = {
        "model": label,
        "q_err_max": float(q_err.max(skipna=True)),
        "q_err_p99": float(q_err.quantile(0.99)),
        "q_err_median": float(q_err.median(skipna=True)),
    }
    return out

checks = []
# Pair mu/q columns by common suffix if possible; otherwise just take first two.
def suffix(c): 
    m = re.search(r"(boxcar|ewls|exp|wls)$", c.lower())
    return m.group(1) if m else c.lower()

q_by = {suffix(c): c for c in q_cols}
for mu in mu_cols:
    suf = suffix(mu)
    if suf in q_by:
        checks.append(report_predictive_check(mu, q_by[suf], suf))
# fallback if nothing matched
if not checks and mu_cols and q_cols:
    checks.append(report_predictive_check(mu_cols[0], q_cols[0], "predictive"))

summary = pd.DataFrame({
    "rho_err_max": [float(rho_err.max(skipna=True))],
    "rho_err_p99": [float(rho_err.quantile(0.99))],
    "c_err_max":   [float(c_err.max(skipna=True))],
    "c_err_p99":   [float(c_err.quantile(0.99))],
})
display(summary)

if checks:
    display(pd.DataFrame(checks))

# --- “First valid row” diagnostics (footnote / indexing sanity) ---
firsts = []
for t, d in g:
    row = {
        "ticker": t,
        "first_gamma_row": int(d[col_gamma].first_valid_index()) if d[col_gamma].notna().any() else None,
        "first_rho_row": int(d[col_rho].first_valid_index()) if d[col_rho].notna().any() else None,
    }
    if mu_cols:
        row["first_mu_row"] = int(d[mu_cols[0]].first_valid_index()) if d[mu_cols[0]].notna().any() else None
    firsts.append(row)
display(pd.DataFrame(firsts).head(20))


,rho_err_max,rho_err_p99,c_err_max,c_err_p99
0,2.220446e-16,1.110223e-16,4.371503e-16,2.220446e-16


,ticker,first_gamma_row,first_rho_row
0,AES,16.0,16.0
1,BA,434.0,434.0
2,C,852.0,852.0
3,F,1270.0,1270.0
4,GE,NaN,NaN
5,JPM,2106.0,2106.0
6,LNC,2524.0,2524.0
7,LOW,2942.0,2942.0
8,LUV,3360.0,3360.0
9,MAR,3778.0,3778.0


This checks that sanity checks passed